<a href="https://colab.research.google.com/github/jyryu3161/chembounce/blob/main/chembounce_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Anaconda installation
# Conda installation
import os
conda_loc = "/usr/local"
os.system("wget https://repo.anaconda.com/archive/Anaconda3-2024.02-1-Linux-x86_64.sh")
os.system("chmod +x Anaconda3-2024.02-1-Linux-x86_64.sh")
os.system(f"bash ./Anaconda3-2024.02-1-Linux-x86_64.sh -b -f -p {conda_loc}")

0

In [ ]:
#@title Environment setting for Chembounce
#@markdown This process takes apprx. 9mins
import os
import sys
# git clone
git_plf_n = 'chembounce'
if not os.path.isdir(git_plf_n) and os.path.split(os.getcwd())[1] != git_plf_n:
    print("Downloading git repository...")
    !git clone https://github.com/jyryu3161/chembounce.git
if os.path.split(os.getcwd())[1] != git_plf_n:
    os.chdir(git_plf_n)
sys.path.append(os.path.abspath("./"))
# environment setting
if not os.path.isfile(f'{conda_loc}/envs/chembounce/bin/python'):
    print("Installing package dependencies...")
    os.system(f"{conda_loc}/bin/conda env create -n chembounce -f ./environment.yml --quiet -y")
# !pip3 install -q --no-warn-conflicts scaffoldgraph==1.1.2 rdkit==2020.09.5 scipy pandas numpy oddt pubchempy molvs

# Dataset
data_installed=False
if os.path.isdir('./data'):
    if len(set(os.listdir('./data')) & set(['fragment_data.pickle','Scaffolds_processed.txt']))==2:
        data_installed = True
if not data_installed:
    print("Downloading data...")
    os.system('bash install.sh')
from google.colab import files

Cloning into 'chembounce'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 97 (delta 48), reused 71 (delta 30), pack-reused 0
Receiving objects: 100% (97/97), 389.70 KiB | 2.33 MiB/s, done.
Resolving deltas: 100% (48/48), done.
Installing package dependencies...


In [ ]:
#@title Input parameters {run:'auto'}
#@markdown ##Structures
#@markdown ####input_smiles
#@markdown #####Input SMILES, the target molecular structure
input_smiles="O=C(C)Oc1ccccc1C(=O)O" #@param {type:'string'}

#@markdown ####core_smiles (optional)
#@markdown #####(Optional) Core SMILES which should not be altered while scaffold hopping
core_smiles="C" #@param {type:'string'}

#@markdown ##Parameters
#@markdown ####top_n
#@markdown #####Number of top fragments to test
top_n=10 #@param {type:'integer'}

#@markdown ####threshold
#@markdown #####Similarity threshold, between 0 and 1: used to exclude irrelated molecular structure, based on the similarity between the original structure and scaffold-hopped one. Default is 0.5
threshold=0.5 #@param {type:'slider',min:0.0, max:1.0, step:0.01}

#@markdown ####output_dir
#@markdown #####Output location
output_dir = './output' #@param {type:'string'}

#@markdown ####low_mem
#@markdown #####Low memory mode (recommended): Note that ordinary version requires at 55GB RAM, but less than 2GB RAM for low memory mode.
low_mem=True #@param {type:'boolean'}


os.makedirs(output_dir,exist_ok=True)
if os.path.isfile(os.path.join(output_dir,'result.txt')):
    print('Warning: A result file already exists')
print(
    f"Input SMILES\t:{input_smiles}",
    f"\nCore SMILES\t:{core_smiles}",
    f"\nThreshold\t:{threshold}",
    f"\nTop fragments\t:{top_n}",
    f"\nLow memory mode\t:{low_mem}",
    f"\nOutput location\t:{os.path.abspath(output_dir)}"
)

Input SMILES	:O=C(C)Oc1ccccc1C(=O)O 
Core SMILES	:C 
Threshold	:0.5 
Top fragments	:10 
Low memory mode	:True 
Output location	:/content/chembounce/output


In [ ]:
#@title Run
#@markdown Once this process is finished, the result will be downloaded : OUTPUT_DIR.tar.gz
print(
    f"Input SMILES\t:{input_smiles}",
    f"\nCore SMILES\t:{core_smiles}",
    f"\nThreshold\t:{threshold}",
    f"\nTop fragments\t:{top_n}",
    f"\nLow memory mode\t:{low_mem}",
    f"\nOutput location\t:{os.path.abspath(output_dir)}"
)
# command line
cmd = "/usr/local/envs/chembounce/bin/python chembounce.py -o %s -i \"%s\" -n %s -t %s -l"
cmd = cmd % (
    os.path.abspath(output_dir),
    input_smiles,
    top_n,
    threshold)
if core_smiles:
  cmd += f' -c \"{core_smiles}\"'
print(cmd)
os.system(cmd)
# # In case to use the function as cell
# import chembounce
# result_df = chembounce.chembounce(
#     target_smiles=input_smiles,
#     fragments_DB=[],
#     core_smiles=core_smiles,
#     threshold=threshold,
#     final_top_n=top_n,
#     output_dir=output_dir,
#     low_mem=low_mem,
#     tqdm_quiet=True,
#     )

os.system(f'tar -czvf {output_dir}.tar.gz {output_dir}')
files.download(f"{output_dir}.tar.gz")
# files.download(os.path.join(output_dir,'result.txt'))

Input SMILES	:O=C(C)Oc1ccccc1C(=O)O 
Core SMILES	:C 
Threshold	:0.5 
Top fragments	:10 
Low memory mode	:True 
Output location	:/content/chembounce/output
/usr/local/envs/chembounce/bin/python chembounce.py -o /content/chembounce/output -i "O=C(C)Oc1ccccc1C(=O)O" -n 10 -t 0.5 -l -c "C"


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>